In [2]:
from hamiltonian import HamiltonianSmall, Hamiltonian

In [3]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

# 12 qubits
lih_jw = Hamiltonian('LiH_STO3g_12qubits', 'jw')
lih_parity = Hamiltonian('LiH_STO3g_12qubits', 'parity')
lih_bk = Hamiltonian('LiH_STO3g_12qubits', 'bk')

# 14 qubits
h2o_jw = Hamiltonian('H2O_STO3g_14qubits', 'jw')
h2o_parity = Hamiltonian('H2O_STO3g_14qubits', 'parity')
h2o_bk = Hamiltonian('H2O_STO3g_14qubits', 'bk')

beh2_jw = Hamiltonian('BeH2_STO3g_14qubits', 'jw')
beh2_parity = Hamiltonian('BeH2_STO3g_14qubits', 'parity')
beh2_bk = Hamiltonian('BeH2_STO3g_14qubits', 'bk')

# 16 qubits
nh3_jw = Hamiltonian('NH3_STO3g_16qubits', 'jw')
nh3_parity = Hamiltonian('NH3_STO3g_16qubits', 'parity')
nh3_bk = Hamiltonian('NH3_STO3g_16qubits', 'bk')

# 20 qubits
c2_jw = Hamiltonian('C2_STO3g_20qubits', 'jw')
c2_parity = Hamiltonian('C2_STO3g_20qubits', 'parity')
c2_bk = Hamiltonian('C2_STO3g_20qubits', 'bk')

In [ ]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk}

In [ ]:
%time h2_jw_4.pauli_rep.ground()
%time h2_jw.pauli_rep.ground()
%time lih_jw.pauli_rep.ground()
%time energy, state = h2o_jw.pauli_rep.ground()
# %time energy, state = nh3_jw.pauli_rep.ground() # takes about 4 minutes

In [ ]:
%time h2_jw_4.pauli_rep.ground(multithread=True)
%time h2_jw.pauli_rep.ground(multithread=True)
%time lih_jw.pauli_rep.ground(multithread=True)
%time energy, state = h2o_jw.pauli_rep.ground(multithread=True)
# %time energy, state = ammonia_jw.pauli_rep.ground(multithread=True) # takes about 1 minute

# Variance formula

\begin{align}
    \Var[\nu] 
    &=
    \sum_{\Qarrow,\Rarrow}
        f_\beta(\Qarrow,\Rarrow)
        \alpha_\Qarrow \alpha_\Rarrow
        \tr(\rho\Qarrow\Rarrow)
    - \tr(\rho H_0 )^2
\end{align}

In [ ]:
ham = h2_jw_4
energy, state = ham.pauli_rep.ground()
β = ham.pauli_rep.local_dists_uniform()
%time ham.pauli_rep.variance_local(energy, state, β)

In [ ]:
print("ell_1: ", ham.pauli_rep.variance_ell_1(energy))

In [ ]:
# this code is from an old idea, which ultimately did not work well.

β = ham.pauli_rep.local_dists_pnorm(1)
print("1_norm: ", ham.pauli_rep.variance_local(energy, state, β))

β = ham.pauli_rep.local_dists_pnorm(2)
print("2_norm: ", ham.pauli_rep.variance_local(energy, state, β))

β = ham.pauli_rep.local_dists_pnorm('infinity')
print("max_norm: ", ham.pauli_rep.variance_local(energy, state, β))

# Variance optimisation (method=diagonal)

This is not the correct optimisation. However it
- gives good results;
- is quicker than the full optimisation problem;
- is convex (so local minimums are global);
- does not need access to the Hartree-Fock bitstring for the encoding.

Diagonal minimisation asks us to find $\{\beta_{i,P}\}$ in order to minimise:
$$
    \sum_{\Qarrow} \alpha_\Qarrow^2 \prod_{i\in\supp(\Qarrow)} \beta_{i,Q_i}^{-1}
    \qquad
    \textrm{subject to}
    \qquad
    \beta_{i,X}+\beta_{i,Y}+\beta_{i,Z}=1 \,\forall i,
    \qquad
    \beta_{i,P}\ge 0
$$

And we have an implementation using Lagrange multipliers

In [ ]:
β = ham.pauli_rep.local_dists_optimal('diagonal', 'scipy')
print(ham.pauli_rep.variance_local(energy, state, β))

In [ ]:
β = ham.pauli_rep.local_dists_optimal('diagonal', 'lagrange')
print(ham.pauli_rep.variance_local(energy, state, β))

In [8]:
# time comparison lih_jw has 12 qubits

ham = lih_jw
%time β_scipy = ham.pauli_rep.local_dists_optimal('diagonal', 'scipy')
%time β_lagrange = ham.pauli_rep.local_dists_optimal('diagonal', 'lagrange')

energy, state = ham.pauli_rep.ground(multithread=True)
var_scipy = ham.pauli_rep.variance_local(energy, state, β_scipy)
var_lagrange = ham.pauli_rep.variance_local(energy, state, β_lagrange)

discrepancy = abs(var_scipy - var_lagrange)
print("discrepancy between variances: ", discrepancy)

CPU times: user 1min 4s, sys: 7.32 s, total: 1min 11s
Wall time: 12 s
CPU times: user 2.25 s, sys: 67.7 ms, total: 2.32 s
Wall time: 1.74 s
discrepancy between variances:  4.931720458856148e-05


# Variance optimisation (method=mixed)

This is the full optimisation problem. It requires access to the Hartree-Fock bitstring $m$ or `bitstring_HF` so that the HF state reads
$\frac1{2^n}\otimes_{i=1}^n (I+m_i Z)$

In the JW encoding these are:
- H2 = `1010` (on four qubits)
- H2 = `10001000`
- LiH = `100000100000`
- H2O = `11111001111100`
- BeH2 = `11100001110000`
- NH3 = `1111100011111000`
You can retrieve them by calling `Hamiltonian.read_bitstring_HF()`

Consider the set of influential pairs:
\begin{align}
    \mathcal{I}_\mathrm{comp}
    =
    \left\{\left.
        (\Qarrow,\Rarrow)
        \,\right|\,
        \textrm{for all $i$, either $Q_i=R_i$, or $\{Q_i,R_i\}=\{I,Z\}$}
    \right\}
\end{align}

Then the cost function to optimise will be:
\begin{align}
    \mathrm{cost}(\{\beta_i\}_{i=1}^n)
    =
    \sum_{\Qarrow,\Rarrow\in\mathcal{I}_\mathrm{comp}}
        \alpha_\Qarrow
        \alpha_\Rarrow
        \prod_{i | Q_i=R_i\neq I}
            \beta_{i,Q_i}^{-1}
        \prod_{i | Q_i\neq R_i}
            m_i
\end{align}

Warning, the small Hamiltonians don't follow the pattern because they use other reduction techniques

In [ ]:
bitstring_HF = ham.read_bitstring_HF()
β = ham.pauli_rep.local_dists_optimal('mixed', 'scipy', bitstring_HF=bitstring_HF)
print(ham.pauli_rep.variance_local(energy, state, β))

# Variance after LDF grouping

We should use 1-norm sampling for $\kappa$

\begin{align}
    \Var[\nu] 
    = 
    \left(
    \sum_{k=1}^{n_c}
        \frac1{\kappa_k} 
        \sum_{\Qarrow,\Rarrow\in C^{(k)}}
            \alpha_\Qarrow\alpha_\Rarrow
            \prod_{i\in\supp(\Qarrow\Rarrow)} \langle \Qarrow\Rarrow \rangle
    \right)
    - \langle H_0 \rangle^2
\end{align}

In [ ]:
from var import variance_ldf, kappa_1norm #kappa_uniform

In [ ]:
ldf = ham.ldf()
kappa = kappa_1norm(ldf)
energy_tf = ham.pauli_rep.energy_tf(energy)
variance_ldf(ldf, state, kappa, energy_tf)

# Benchmarking

In [ ]:
def variances_dict(ham, β_diag=None, β_mix=None):
    pr = ham.pauli_rep
    dic = {}
    
    energy, state = pr.ground(multithread=True)
    print("energy :", energy)

    # ell_1
    var = pr.variance_ell_1(energy)
    print("ell 1: ", var)
    dic['ell_1'] = var
    
    # LDF with 1-norm sampling
    ldf = ham.ldf()
    kappa = kappa_1norm(ldf)
    energy_tf = pr.energy_tf(energy)
    var = variance_ldf(ldf, state, kappa, energy_tf)
    print("ldf 1norm: ", var)
    dic['ldf_1norm'] = var
    
    # uniform
    β_uniform = pr.local_dists_uniform()
    var = pr.variance_local(energy, state, β_uniform, multithread=True)
    print("uniform: ", var)
    dic['uniform'] = var
    
    # optimal (diagonal)
    if β_diag is not None:
        var = pr.variance_local(energy, state, β_diag, multithread=True)
        print("optimal diagonal: ", var)
        dic['optimal_diag'] = var
 
    # optimal (mixed)
    if β_mix is not None:
        var = pr.variance_local(energy, state, β_mix, multithread=True)
        print("optimal mixed: ", var)
        dic['optimal_mix'] = var
    
    return dic

from matplotlib import pyplot as plt

def variances_graph(variances):
    num_variances = len(variances)
    x = range(num_variances)
    height = list(variances.values())

    plt.bar(x, height)
    plt.xticks(x, list(variances.keys()), rotation=20)
    plt.title(title)

    plt.show()

In [ ]:
variances_ALL = {}
beta_optimal_ALL = {}

In [ ]:
name = 'h2_jw_4'
ham = h2_jw_4
bitstring_HF = ham.read_bitstring_HF()

title = "Variances for various algorithms on H2 in JW encoding over 4 qubits"

%time β_diag = ham.pauli_rep.local_dists_optimal('diagonal')
%time β_mix = ham.pauli_rep.local_dists_optimal('mixed', bitstring_HF)
beta_optimal_ALL[name] = {'diagonal': β_diag, 'mixed': β_mix}

%time variances_ALL[name] = variances_dict(ham, β_diag=β_diag, β_mix=β_mix)

print("=====")
print(title)
print("=====")
variances_ALL[name]

In [ ]:
variances_graph(variances_ALL[name])

In [ ]:
name = 'h2_jw'
ham = h2_jw
bitstring_HF = ham.read_bitstring_HF()

title = "Variances for various algorithms on H2 in JW encoding over 8 qubits"

%time β_diag = ham.pauli_rep.local_dists_optimal('diagonal')
%time β_mix = ham.pauli_rep.local_dists_optimal('mixed', bitstring_HF)
beta_optimal_ALL[name] = {'diagonal': β_diag, 'mixed': β_mix}

%time variances_ALL[name] = variances_dict(ham, β_diag=β_diag, β_mix=β_mix)

print("=====")
print(title)
print("=====")
variances_ALL[name]

In [ ]:
variances_graph(variances_ALL[name])

In [ ]:
name = 'lih_jw'
ham = lih_jw
bitstring_HF = ham.read_bitstring_HF()

title = "Variances for various algorithms on LiH in JW encoding over 12 qubits"

%time β_diag = ham.pauli_rep.local_dists_optimal('diagonal')
%time β_mix = ham.pauli_rep.local_dists_optimal('mixed', bitstring_HF)
beta_optimal_ALL[name] = {'diagonal': β_diag, 'mixed': β_mix}

%time variances_ALL[name] = variances_dict(ham, β_diag=β_diag, β_mix=β_mix)

print("=====")
print(title)
print("=====")
variances_ALL[name]

In [ ]:
variances_graph(variances_ALL[name])

In [ ]:
name = 'h2o_jw'
ham = h2o_jw
bitstring_HF = ham.read_bitstring_HF()

title = "Variances for various algorithms on H2O in JW encoding over 14 qubits"

%time β_diag = ham.pauli_rep.local_dists_optimal('diagonal')
%time β_mix = ham.pauli_rep.local_dists_optimal('mixed', bitstring_HF)
beta_optimal_ALL[name] = {'diagonal': β_diag, 'mixed': β_mix}

%time variances_ALL[name] = variances_dict(ham, β_diag=β_diag, β_mix=β_mix)

print("=====")
print(title)
print("=====")
variances_ALL[name]

In [ ]:
variances_graph(variances_ALL[name])

In [ ]:
# mac book pro is too weak for this!

#name = 'nh3_jw'
#ham = nh3_jw
#bitstring_HF = ham.read_bitstring_HF()

#title = "Variances for various algorithms on ammonia in JW encoding over 14 qubits"

#%time β_diag = ham.pauli_rep.local_dists_optimal('diagonal')
# rudy needs to help me! this will take too long using my pedestrian approach
#%time β_mix = ham.pauli_rep.local_dists_optimal('mixed', bitstring_HF)
#beta_optimal_ALL[name] = {'diagonal': β_diag, 'mixed': β_mix}

#%time variances_ALL[name] = variances_dict(ham, β_diag=β_diag)#, β_mix=β_mix)

#print("=====")
#print(title)
#print("=====")
#variances_ALL[name]